# TMA4320 Industrial Mathematics Project
### Oliver Ruden, Åsmund Mjøs & Astrid Mysterud

Introduksjon

## Oppgave 1

1.1) La $a=31$, $b=7$ og $c=1$. Da blir $d=a\cdot b+c=218$, slik at $x=[3,1,7,0,1,2,1]$ og $y=[2,1,8]$. Modellen gir oss $\hat{z}=[\hat{z}_0, \ldots, \hat{z}_6]=f_\theta([3,1,7,0,1,2,1])$. Vi ønsker å finne $\theta$ slik at $\hat{y}=[\hat{z}_4, \hat{z}_5, \hat{z}_6]=[2,1,8]$.

1.2) Vi har at

\begin{align*}
    x^{(0)}=[3,1,7,0,1]\quad\quad[\hat{z}_0^{(0)}, \ldots, \hat{z}_4^{(0)}]=f_\theta(x^{(0)})\\
    x^{(1)}=[3,1,7,0,1,\hat{z}_4^{(0)}]\quad\quad[\hat{z}_0^{(1)}, \ldots, \hat{z}_5^{(1)}]=f_\theta(x^{(1)})\\
    x^{(2)}=[3,1,7,0,1,\hat{z}_4^{(0)}, \hat{z}_5^{(1)}]\quad\quad[\hat{z}_0^{(2)}, \ldots, \hat{z}_6^{(2)}]=f_\theta(x^{(2)})\\
    x^{(3)}=[3,1,7,0,1,\hat{z}_4^{(0)}, \hat{z}_5^{(1)}, \hat{z}_6^{(2)}]
\end{align*}
Da får vi $\hat{y}=[\hat{z}_3^{(0)}, \hat{z}_4^{(1)}, \hat{z}_5^{(2)}]$ som vi kan sammenligne med $y=[2,1,8]$.

1.3)
La oss anta at vi starter med en generell matrise:
\begin{equation*}
    \hat{Y}=\left[
        \begin{array}{cccc}
            a_{00} & \cdots & a_{03}\\
            \vdots & \ddots & \vdots\\
            a_{40} & \cdots & a_{43}
        \end{array}
        \right]    
\end{equation*}
Dette gir:
\begin{equation*}
    p = \left[\begin{array}{cccc}
        a_{40} & a_{31} & a_{22} & a_{13}
        \end{array}
        \right]
\end{equation*}
Og vi får:
\begin{equation*}
    \mathcal{L}(\theta, \mathcal{D})=\frac{1}{4}\sum_{i=0}^{3} -\log(p_i)\geq 0
\end{equation*}
Hvor vi har likhet når:
\begin{equation*}
    a_{40} = a_{31} = a_{22} = a_{13} = 1
\end{equation*}
Fordi vi har at $\hat{Y}$ er en sannsynlighetsfordeling, må resten av elementene være lik null, og vi får:
\begin{equation*}
    \hat{Y}=Y=\text{onehot}(y)=\left[
        \begin{array}{cccc}
            0 & 0 & 0 & 0\\
            0 & 0 & 0 & 1\\
            0 & 0 & 1 & 0\\
            0 & 1 & 0 & 0\\
            1 & 0 & 0 & 0
        \end{array}
        \right]    
\end{equation*}
Dette gir:
\begin{equation*}
    \hat{y} = [4,\;3,\;2,\;1]
\end{equation*}